Install libraries

In [1]:
%pip install requests
%pip install pandas
%pip install numpy
%pip install scikit-learn

Defaulting to user installation because normal site-packages is not writeable

[notice] A new release of pip is available: 23.0.1 -> 24.0
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.
Defaulting to user installation because normal site-packages is not writeable

[notice] A new release of pip is available: 23.0.1 -> 24.0
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.
Defaulting to user installation because normal site-packages is not writeable

[notice] A new release of pip is available: 23.0.1 -> 24.0
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.
Defaulting to user installation because normal site-packages is not writeable

[notice] A new release of pip is available: 23.0.1 -> 24.0
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to us

In [2]:
import requests
import pandas as pd
import numpy as np
from sklearn.cluster import DBSCAN

Get hourly weather data with ETL framework

In [3]:
parameters = {"ids": "KSBA",
              "format": "json",
              "taf": "false",
              "hours": 1000}
url = "https://aviationweather.gov/api/data/metar"
data = requests.get(url, params = parameters).json()
df = pd.json_normalize(data)
df.head()

,metar_id,icaoId,receiptTime,obsTime,reportTime,temp,dewp,wdir,wspd,wgst,...,vertVis,metarType,rawOb,mostRecent,lat,lon,elev,prior,name,clouds
0,539358857,KSBA,2024-05-02 20:58:10,1714683180,2024-05-02 21:00:00,18.3,13.3,230,7,NaN,...,NaN,METAR,KSBA 022053Z 23007KT 9SM FEW020 18/13 A2990 RM...,1,34.4241,-119.842,2,3,"Santa Barbara Muni, CA, US","[{'cover': 'FEW', 'base': 2000}]"
1,539336838,KSBA,2024-05-02 19:58:04,1714679580,2024-05-02 20:00:00,17.8,13.3,160,7,NaN,...,NaN,METAR,KSBA 021953Z 16007KT 6SM HZ SCT018 18/13 A2992...,0,34.4241,-119.842,2,3,"Santa Barbara Muni, CA, US","[{'cover': 'SCT', 'base': 1800}]"
2,539315950,KSBA,2024-05-02 18:58:20,1714675980,2024-05-02 19:00:00,16.7,12.8,150,7,NaN,...,NaN,METAR,KSBA 021853Z 15007KT 6SM HZ FEW017 17/13 A2993...,0,34.4241,-119.842,2,3,"Santa Barbara Muni, CA, US","[{'cover': 'FEW', 'base': 1700}]"
3,539292927,KSBA,2024-05-02 17:58:07,1714672380,2024-05-02 18:00:00,16.7,12.8,160,6,NaN,...,NaN,METAR,KSBA 021753Z 16006KT 9SM SCT017 17/13 A2994 RM...,0,34.4241,-119.842,2,3,"Santa Barbara Muni, CA, US","[{'cover': 'SCT', 'base': 1700}]"
4,539271619,KSBA,2024-05-02 16:58:19,1714668780,2024-05-02 17:00:00,15.6,12.8,150,5,NaN,...,NaN,METAR,KSBA 021653Z 15005KT 8SM OVC012 16/13 A2994 RM...,0,34.4241,-119.842,2,3,"Santa Barbara Muni, CA, US","[{'cover': 'OVC', 'base': 1200}]"


Convert observation time and temperature into UTC and Fahrenheit units

In [4]:
df["UTC"] = pd.to_datetime(df["obsTime"], unit = "s")
df["Temperature"] = np.round(np.array(df["temp"])*1.8 + 32, 0)
df = df.drop(["clouds","obsTime","temp"], axis = 1)
df.head()

,metar_id,icaoId,receiptTime,reportTime,dewp,wdir,wspd,wgst,visib,altim,...,metarType,rawOb,mostRecent,lat,lon,elev,prior,name,UTC,Temperature
0,539358857,KSBA,2024-05-02 20:58:10,2024-05-02 21:00:00,13.3,230,7,NaN,9,1012.6,...,METAR,KSBA 022053Z 23007KT 9SM FEW020 18/13 A2990 RM...,1,34.4241,-119.842,2,3,"Santa Barbara Muni, CA, US",2024-05-02 20:53:00,65.0
1,539336838,KSBA,2024-05-02 19:58:04,2024-05-02 20:00:00,13.3,160,7,NaN,6,1013.3,...,METAR,KSBA 021953Z 16007KT 6SM HZ SCT018 18/13 A2992...,0,34.4241,-119.842,2,3,"Santa Barbara Muni, CA, US",2024-05-02 19:53:00,64.0
2,539315950,KSBA,2024-05-02 18:58:20,2024-05-02 19:00:00,12.8,150,7,NaN,6,1013.6,...,METAR,KSBA 021853Z 15007KT 6SM HZ FEW017 17/13 A2993...,0,34.4241,-119.842,2,3,"Santa Barbara Muni, CA, US",2024-05-02 18:53:00,62.0
3,539292927,KSBA,2024-05-02 17:58:07,2024-05-02 18:00:00,12.8,160,6,NaN,9,1014.0,...,METAR,KSBA 021753Z 16006KT 9SM SCT017 17/13 A2994 RM...,0,34.4241,-119.842,2,3,"Santa Barbara Muni, CA, US",2024-05-02 17:53:00,62.0
4,539271619,KSBA,2024-05-02 16:58:19,2024-05-02 17:00:00,12.8,150,5,NaN,8,1014.0,...,METAR,KSBA 021653Z 15005KT 8SM OVC012 16/13 A2994 RM...,0,34.4241,-119.842,2,3,"Santa Barbara Muni, CA, US",2024-05-02 16:53:00,60.0


In [5]:
x = np.array(df["Temperature"]).reshape(-1, 1)

In [6]:
clusters = DBSCAN(eps = 1, min_samples = 24).fit_predict(x)

In [7]:
df = df[["UTC","Temperature"]].assign(Anomaly = clusters)
df.head()

,UTC,Temperature,Anomaly
0,2024-05-02 20:53:00,65.0,0
1,2024-05-02 19:53:00,64.0,0
2,2024-05-02 18:53:00,62.0,0
3,2024-05-02 17:53:00,62.0,0
4,2024-05-02 16:53:00,60.0,0


In [8]:
df.loc[(df["Anomaly"] == -1),:]

,UTC,Temperature,Anomaly
29,2024-05-01 23:53:00,66.0,-1
33,2024-05-01 19:53:00,66.0,-1
71,2024-04-30 07:53:00,70.0,-1
72,2024-04-30 06:53:00,72.0,-1
73,2024-04-30 05:53:00,73.0,-1
74,2024-04-30 05:22:00,73.0,-1
80,2024-04-29 23:53:00,66.0,-1
81,2024-04-29 22:53:00,69.0,-1
82,2024-04-29 21:53:00,67.0,-1
83,2024-04-29 20:53:00,66.0,-1
